# Create a Graph from a text

This notebook demonstrates how to extract graph from any text using the graph maker

Steps:
- Define an Ontology
- Load a list of example text chunks. We will use the Lord of the Rings summary from this wikipedia page. 
- Create Graph using an Open source model using Groq APIs. 
- Save the graph to Neo4j db
- Visualise



Loading the graph maker functions ->

In [411]:
import os

In [412]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = "TEST"

In [413]:
from knowledge_graph_maker import GraphMaker, Ontology, OpenAIClient
from knowledge_graph_maker import Document

# Define the Ontology. 

The ontology is a pydantic model with the following schema. 

```python
class Ontology(BaseModel):
    label: List[Union[str, Dict]]
    relationships: List[str]
```



In [414]:
from lotr_wikipedia_summary import lord_of_the_rings_wikipedia_summary as example_text_list

Here is the ontology we will use for the eldenring summaries ->

In [415]:
ontology = Ontology(
    labels=[
        {"NPC": "Person name without any adjectives, Remember a person may be referenced by their name or using a pronoun"},
        {"Object": "Do not add the definite article 'the' in the object name"},
        {"Event": "Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc."},
        {"Place": "Locations where specific events took place"},
        "Faction",
        {"Miscellaneous": "Any important concept can not be categorised with any other given label"},
    ],
    relationships=[
        "Relation between any pair of Entities"
        ],
)

In [416]:
# ontology = Ontology(
#     labels=[
#         {"Person": "Person name without any adjectives, Remember a person may be referenced by their name or using a pronoun"},
#         {"Object": "Do not add the definite article 'the' in the object name"},
#         {"Event": "Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc."},
#         {"Place": "Locations where specific events took place"},
#         "Document",
#         "Faction",
#         "Action",
#         {"Miscellaneous": "Any important concept can not be categorised with any other given label"},
#     ],
#     relationships=[
#         "Relation between any pair of Entities"
#         ],
# )

## Select a Model

Groq support the following models at present. 

*LLaMA3 8b*
Model ID: llama3-8b-8192

*LLaMA3 70b*
Model ID: llama3-70b-8192

*Mixtral 8x7b*
Model ID: mixtral-8x7b-32768

*Gemma 7b*
Model ID: gemma-7b-it


Selecting a model for this example ->


In [417]:

## Groq models
model = "mixtral-8x7b-32768"
# model ="llama3-8b-8192"
# model = "llama3-70b-8192"
# model="gemma-7b-it"

## Open AI models
oai_model="gpt-4o-mini"

## Use Groq
# llm = GroqClient(model=model, temperature=0.1, top_p=0.5)
## OR Use OpenAI
llm = OpenAIClient(model=oai_model, temperature=0.1, top_p=0.5)


## Create documents out of text chumks. 
Documents is a pydantic model with the following schema 

```python
class Document(BaseModel):
    text: str
    metadata: dict
```

The metadata we add to the document here is copied to every relation that is extracted out of the document. More often than not, the node pairs have multiple relation with each other. The metadata helps add more context to these relations

In this example I am generating a summary of the text chunk, and the timestamp of the run, to be used as metadata. 


In [418]:
#from langchain_experimental.text_splitter import SemanticChunker
# from langchain_openai.embeddings import OpenAIEmbeddings

In [419]:
# text_splitter = SemanticChunker(OpenAIEmbeddings())
# docs1 = text_splitter.split_text(temp)

In [420]:
from langchain import hub
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel
from typing import Optional, List

In [421]:
obj = hub.pull("wfh/proposal-indexing")
chunking_llm = ChatOpenAI(model='gpt-4o-mini', openai_api_key = os.getenv("OPENAI_API_KEY", 'YouKey'))

/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [422]:
# use it in a runnable
runnable = obj | chunking_llm

In [423]:
# Pydantic data class
class Sentences(BaseModel):
    sentences: List[str]
    
# Extraction
extraction_chain = create_extraction_chain_pydantic(pydantic_schema=Sentences, llm=chunking_llm)

In [424]:
def get_propositions(text):
    runnable_output = runnable.invoke({
    	"input": text
    }).content
    
    propositions = extraction_chain.run(runnable_output)[0].sentences
    return propositions

In [425]:
with open('./data_input/eldenring.txt') as file:
    essay = file.read()

In [426]:
paragraphs = essay.split("\n\n")

In [427]:
len(paragraphs)

60

In [436]:
essay_propositions = []

for para in paragraphs:
    propositions = get_propositions(para)
    
    essay_propositions.extend(propositions)
    # print (f"Done with {i}")


# for i, para in enumerate(paragraphs[:5]):
#     propositions = get_propositions(para)
    
#     essay_propositions.extend(propositions)
#     print (f"Done with {i}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=Pydan

In [437]:
print (f"You have {len(essay_propositions)} propositions")
essay_propositions

You have 556 propositions


['Lore in Elden Ring covers all the information related to the world of Elden Ring.',
 'Lore in Elden Ring covers all the information related to the mythos of Elden Ring.',
 'Lore in Elden Ring includes past events that occurred long ago.',
 'Lore in Elden Ring includes the history of the Lands Between.',
 'Lore in Elden Ring includes the mystery of the elusive Elden Ring.',
 'All information related to the world, mythos, past events, history, and mystery can be found in the lore of Elden Ring.',
 'This page contains heavy spoilers.',
 'The Mythos of Elden Ring was written by George R. R. Martin.',
 'Elden Ring is a video game.',
 'Elden Ring has a lore overview.',
 'The storyteller folds her slender hands.',
 'The storyteller folds both pairs of her hands.',
 'The storyteller speaks.',
 'The storyteller states that the events happened an age ago.',
 'The storyteller recalls the events as true.',
 'The Lands Between is the world of Elden Ring.',
 'Limgrave is one of the areas in the ga

In [438]:
import datetime
current_time = str(datetime.datetime.now())

In [439]:
graph_maker = GraphMaker(ontology=ontology, llm_client=llm, verbose=False)

In [440]:
def generate_summary(text):
    SYS_PROMPT = (
        "Succintly summarise the text provided by the user. "
        "Respond only with the summary and no other comments"
    )
    try:
        summary = llm.generate(user_message=text, system_message=SYS_PROMPT)
    except:
        summary = ""
    finally:
        return summary

In [441]:
docs = map(
    lambda t: Document(text=t, metadata={"summary": generate_summary(t), 'generated_at': current_time}),
    essay_propositions
)

## Create Graph
Finally run the Graph Maker to generate graph. 

In [442]:
graph = graph_maker.from_documents(
    list(docs), 
    delay_s_between=0 ## delay_s_between because otherwise groq api maxes out pretty fast. 
    ) 
print("Total number of Edges", len(graph))

Using Model:  gpt-4o-mini


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  gpt-4o-mini


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  gpt-4o-mini


...

Using Model:  gpt-4o-mini


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-08-24 20:48:55 - INFO 
LLM Response:
```json
[]
```


▶︎ GRAPH MAKER LOG - 2024-08-24 20:48:55 - INFO 
Trying JSON Parsing: 
```json
[]
```


▶︎ GRAPH MAKER ERROR - 2024-08-24 20:48:55 - INFO 
JSON Parsing failed with error: Expecting value


▶︎ GRAPH MAKER VERBOSE - 2024-08-24 20:48:55 - INFO 
FAULTY JSON: ```json
[]
```


▶︎ GRAPH MAKER LOG - 2024-08-24 20:48:55 - INFO 
Trying Manual Parsing: 
```json
[]
```


▶︎ GRAPH MAKER ERROR - 2024-08-24 20:48:55 - INFO 
Failed to Parse the Edge: {json}
Expecting property name enclosed in double quotes


▶︎ GRAPH MAKER VERBOSE - 2024-08-24 20:48:55 - INFO 
FAULTY EDGE: {json}


▶︎ GRAPH MAKER LOG - 2024-08-24 20:48:55 - INFO 
Manually exracted 0 Edges


▶︎ GRAPH MAKER LOG - 2024-08-24 20:48:55 - INFO 
Document: 556


▶︎ GRAPH MAKER VERBOSE - 2024-08-24 20:48:55 - INFO 
Using Ontology:
labels=[{'Person': 'Person name without 

Using Model:  gpt-4o-mini


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-08-24 20:48:55 - INFO 
LLM Response:
```json
[]
```


▶︎ GRAPH MAKER LOG - 2024-08-24 20:48:55 - INFO 
Trying JSON Parsing: 
```json
[]
```


▶︎ GRAPH MAKER ERROR - 2024-08-24 20:48:55 - INFO 
JSON Parsing failed with error: Expecting value


▶︎ GRAPH MAKER VERBOSE - 2024-08-24 20:48:55 - INFO 
FAULTY JSON: ```json
[]
```


▶︎ GRAPH MAKER LOG - 2024-08-24 20:48:55 - INFO 
Trying Manual Parsing: 
```json
[]
```


▶︎ GRAPH MAKER ERROR - 2024-08-24 20:48:55 - INFO 
Failed to Parse the Edge: {json}
Expecting property name enclosed in double quotes


▶︎ GRAPH MAKER VERBOSE - 2024-08-24 20:48:55 - INFO 
FAULTY EDGE: {json}


▶︎ GRAPH MAKER LOG - 2024-08-24 20:48:55 - INFO 
Manually exracted 0 Edges



Total number of Edges 893


In [ ]:
# for edge in graph:
#     print(edge.model_dump)
#     print(edge.model_dump(exclude=['metadata']), "\n\n")

<bound method BaseModel.model_dump of Edge(node_1=Node(label='Miscellaneous', name='Lore'), node_2=Node(label='Object', name='Elden Ring'), relationship='Lore contains information related to the world and mythos of Elden Ring.', metadata={'summary': "Elden Ring's lore encompasses the world's information and mythos.", 'generated_at': '2024-08-24 20:11:45.149255'}, order=0)>
{'node_1': {'label': 'Miscellaneous', 'name': 'Lore'}, 'node_2': {'label': 'Object', 'name': 'Elden Ring'}, 'relationship': 'Lore contains information related to the world and mythos of Elden Ring.', 'order': 0} 


<bound method BaseModel.model_dump of Edge(node_1=Node(label='Event', name='past events'), node_2=Node(label='Miscellaneous', name='lore'), relationship='The past events are part of the lore in Elden Ring.', metadata={'summary': "Elden Ring's lore includes events from the distant past.", 'generated_at': '2024-08-24 20:11:45.149255'}, order=1)>
{'node_1': {'label': 'Event', 'name': 'past events'}, 'node_2':

# Save the Graph to Neo4j 

In [443]:
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "12345678"
os.environ["NEO4J_URI"]= "bolt://localhost:7687"

In [444]:
from knowledge_graph_maker import Neo4jGraphModel

create_indices = False
neo4j_graph = Neo4jGraphModel(edges=graph, create_indices=create_indices)
neo4j_graph.save()

893